## [RQ2] 
Is there a home-field advantage? It is generally believed that there is an underlying home field advantage in sport, i.e. an highest probability of winning of the home team. Let's check for this, and see whether the outcome of the game (win, draw, lose) is correlated to the playing side (home or away). For 5 different teams of Premier League, show the contingency table (outcome x side) and perform the Chi-squared test to see whether they are correlated or not. State clearly the tested hypothesis and whether it is accepted or rejected.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import chi2_contingency

In [2]:
data = pd.read_json(r"matches\matches_England.json")
data.head()

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364
3,Played,4405654,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",181150,2018-05-13 14:00:00,1612,Anfield,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385704, 'role': 'referee'}, {'r...",Regular,364
4,Played,4405654,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"Manchester United - Watford, 1 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381853, 'role': 'referee'}, {'r...",Regular,364


In [3]:
data=data[["gameweek","winner","label"]]
data.head()

,gameweek,winner,label
0,38,1659,"Burnley - AFC Bournemouth, 1 - 2"
1,38,1628,"Crystal Palace - West Bromwich Albion, 2 - 0"
2,38,1609,"Huddersfield Town - Arsenal, 0 - 1"
3,38,1612,"Liverpool - Brighton & Hove Albion, 4 - 0"
4,38,1611,"Manchester United - Watford, 1 - 0"


In [4]:
#seperating host and away team
x=[]
for i in range(data.shape[0]):
    x.append([data["label"].str.split(",")[i][0].split("-")[0].rstrip(),data["label"].str.split(",")[i][0].split("-")[1].lstrip()])
    

In [5]:
x=pd.DataFrame(x)
x.columns=["host","away"]
data=pd.concat([x,data],axis=1)
data.drop("label",axis=1,inplace=True)
data.head()

,host,away,gameweek,winner
0,Burnley,AFC Bournemouth,38,1659
1,Crystal Palace,West Bromwich Albion,38,1628
2,Huddersfield Town,Arsenal,38,1609
3,Liverpool,Brighton & Hove Albion,38,1612
4,Manchester United,Watford,38,1611


In [6]:
teamid=pd.read_json("teams.json")
teamid=teamid[["name","wyId"]]
teamid.head()

,name,wyId
0,Newcastle United,1613
1,Celta de Vigo,692
2,Espanyol,691
3,Deportivo Alav\u00e9s,696
4,Levante,695


In [7]:
# Finding host team, teamid  by merging team data
mergeddata=data.merge(teamid,left_on="host",right_on="name")
mergeddata.drop("name",axis=1,inplace=True)
mergeddata.rename(columns={"wyId":"hostId"},inplace=True)
mergeddata.head()

,host,away,gameweek,winner,hostId
0,Burnley,AFC Bournemouth,38,1659,1646
1,Burnley,Brighton & Hove Albion,36,0,1646
2,Burnley,Chelsea,31,1610,1646
3,Burnley,Leicester City,34,1646,1646
4,Burnley,Everton,29,1646,1646


In [8]:
# Finding away team, teamid  by merging team data
mergeddata=mergeddata.merge(teamid,left_on="away",right_on="name")
mergeddata.drop("name",axis=1,inplace=True)
mergeddata.rename(columns={"wyId":"awayId"})
mergeddata

,host,away,gameweek,winner,hostId,wyId
0,Burnley,AFC Bournemouth,38,1659,1646,1659
1,Crystal Palace,AFC Bournemouth,16,0,1628,1659
2,Huddersfield Town,AFC Bournemouth,27,1673,1673,1659
3,Liverpool,AFC Bournemouth,34,1612,1612,1659
4,Manchester United,AFC Bournemouth,17,1611,1611,1659
...,...,...,...,...,...,...
375,AFC Bournemouth,Burnley,14,1646,1659,1646
376,Watford,Burnley,33,1646,1644,1646
377,West Bromwich Albion,Burnley,32,1646,1627,1646
378,Stoke City,Burnley,35,0,1639,1646


In [9]:
mergeddata.sort_values(by="gameweek",inplace=True)
mergeddata.reset_index(drop=True,inplace=True)
mergeddata.head()

,host,away,gameweek,winner,hostId,wyId
0,Southampton,Swansea City,1,0,1619,10531
1,Arsenal,Leicester City,1,1609,1609,1631
2,Manchester United,West Ham United,1,1611,1611,1633
3,Brighton & Hove Albion,Manchester City,1,1625,1651,1625
4,Watford,Liverpool,1,0,1644,1612


In [10]:
#selecting five arbitarary team
five_team=pd.unique(mergeddata["host"])[:5]
five_team=teamid[teamid["name"].isin(five_team)].values
five_team

array([['Brighton & Hove Albion', 1651],
       ['Watford', 1644],
       ['Southampton', 1619],
       ['Manchester United', 1611],
       ['Arsenal', 1609]], dtype=object)

In [11]:
#creating empty contingency table
contingencytable=pd.DataFrame(columns=["team","teamid","hostwin","awaywin","hostdraw","awaydraw",
                                      "hostlose","awaylost"])
contingencytable

,team,teamid,hostwin,awaywin,hostdraw,awaydraw,hostlose,awaylost


In [17]:
#seperating each team hostwin ,awaywin ,hostdraw ,awaydraw ,hostlose ,awaylost and
#adding them into contigency table
for team,teamnumber in list(five_team):
    interestofdata=mergeddata[(mergeddata["host"]==team) | (mergeddata["away"]==team)]
    hostwin=interestofdata[(interestofdata["winner"]==teamnumber)&
                           (teamnumber==interestofdata["hostId"]) &
                           ].shape[0]
    awaywin=interestofdata[(interestofdata["winner"]==teamnumber)&
                           (teamnumber==interestofdata["wyId"])].shape[0]
#    print(hostwin,awaywin,interestofdata[(interestofdata["winner"]==teamnumber)].shape[0])
    hostdraw=interestofdata[(interestofdata["winner"]==0)&
                           (teamnumber==interestofdata["hostId"])].shape[0]
    awaydraw=interestofdata[(interestofdata["winner"]==0)&
                           (teamnumber==interestofdata["wyId"])].shape[0]
#     print(hostdraw,awaydraw,)
    hostlose=interestofdata[(~interestofdata["winner"].isin([0,teamnumber]))&
                            (interestofdata["hostId"]==teamnumber)].shape[0]
    awaylose=interestofdata[(~interestofdata["winner"].isin([0,teamnumber]))&
                           (interestofdata["wyId"]==teamnumber)].shape[0]
    
    contingencytable=contingencytable.append(
        pd.Series([team,teamnumber,hostwin,awaywin,hostdraw,awaydraw,hostlose,awaylose],
                  index=contingencytable.columns),ignore_index=True)


 ## ContingencyTable :

In [18]:
contingencytable.set_index("team",inplace=True)
contingencytable.drop(["teamid"],inplace=True,axis=1)
contingencytable

,hostwin,awaywin,hostdraw,awaydraw,hostlose,awaylost
team,,,,,,
Brighton & Hove Albion,7,2,8,6,4,11
Watford,6,4,7,2,6,13
Southampton,4,3,7,8,8,8
Manchester United,15,10,2,4,2,5
Arsenal,15,4,2,4,2,11


In [20]:
chi2_contingency(contingencytable)

(41.42955294782307,
 0.003279575249382425,
 20,
 array([[9.4, 4.6, 5.2, 4.8, 4.4, 9.6],
        [9.4, 4.6, 5.2, 4.8, 4.4, 9.6],
        [9.4, 4.6, 5.2, 4.8, 4.4, 9.6],
        [9.4, 4.6, 5.2, 4.8, 4.4, 9.6],
        [9.4, 4.6, 5.2, 4.8, 4.4, 9.6]]))